<a href="https://colab.research.google.com/github/HarlinLee/science4cast/blob/main/calculate_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
from scipy import sparse
import numpy as np
import pandas as pd
import pickle
from datetime import date, datetime
import time
import networkx as nx
from networkx.algorithms.centrality import katz_centrality
import json
import os

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

from tensorflow import keras

DRIVE_PATH = "./"
day_origin = date(1990,1,1)
NUM_OF_VERTICES = 64719
data_source = os.path.join(DRIVE_PATH,'competition_data', 'CompetitionSet2017_3.pkl')

In [ ]:
def get_sparse_from_year(yy, edges, days=None):
    if not days:
        days = (date(yy,12,31)-day_origin).days
        
    return get_sparse_from_edges(edges[edges[:,2] < days])

def get_sparse_from_edges(edges):
    adj = sparse.csr_matrix(
        (np.ones(2*len(edges)), 
        (
            np.hstack((edges[:,0], edges[:,1])),
            np.hstack((edges[:,1], edges[:,0]))
        )), 
        shape = (NUM_OF_VERTICES,NUM_OF_VERTICES),
        dtype = np.uint16
    )

    adj.setdiag(0) 
    adj.eliminate_zeros()
    
    return adj.asfptype()

def get_edges_from_sparse(mat):
    edges = sparse.triu(mat).nonzero()
    return np.array([edges[0], edges[1]]).T

def orgainze_edges(edges):
    return get_edges_from_sparse(get_sparse_from_edges(edges))

def create_training_data(full_graph,year_start,years_delta,edges_used=500000,vertex_degree_cutoff=10):
    """
    :param full_graph: Full graph, numpy array dim(n,3) [vertex 1, vertex 2, time stamp]
    :param year_start: year of graph
    :param years_delta: distance for prediction in years (prediction on graph of year_start+years_delta)
    :param edges_used: optional filter to create a random subset of edges for rapid prototyping (default: 500,000)
    :param vertex_degree_cutoff: optional filter, for vertices in training set having a minimal degree of at least vertex_degree_cutoff  (default: 10)
    :return:

    all_edge_list: graph of year_start, numpy array dim(n,2)
    unconnected_vertex_pairs: potential edges for year_start+years_delta
    unconnected_vertex_pairs_solution: numpy array with integers (0=unconnected, 1=connected), solution, length = len(unconnected_vertex_pairs)
    """

    years = [year_start,year_start+years_delta]    
    adjs, days = [], []

    for yy in years:
        print('    Create Graph for ', yy)
        days_curr = (date(yy,12,31)-day_origin).days
        days.append(days_curr)
        
        adj = get_sparse_from_year(yy, full_graph, days_curr)
        adjs.append(adj)
        
        print('    num of edges: ', adj.count_nonzero()//2)

    ## Create all edges to be predicted
    all_degs = np.array(adjs[0].sum(0))[0]
    all_vertices = np.array(range(NUM_OF_VERTICES))
    vertex_large_degs = all_vertices[all_degs>=vertex_degree_cutoff] 
    # use only vertices with degrees larger than vertex_degree_cutoff.
    
    ## get all positive examples
    all_edges_after = full_graph[(days[0]<=full_graph[:,2]) & (full_graph[:,2]<days[-1])]
    all_edges_after = all_edges_after[np.all(np.isin(all_edges_after[:,:2], vertex_large_degs), axis=1)]    
    all_edges_after = orgainze_edges(all_edges_after)
    
    print(len(all_edges_after))
    
    ## get some negative examples
    unconnected_vertex_pairs = []
    np.random.seed(0)
    
    while len(unconnected_vertex_pairs) < max(edges_used-len(all_edges_after), len(all_edges_after)):        
        v1,v2 = np.random.choice(vertex_large_degs, 2)

        if (v1 != v2) and (not adjs[0][v1,v2]) and (not adjs[-1][v1,v2]):
            unconnected_vertex_pairs.append((v1,v2))
            
    unconnected_vertex_pairs = orgainze_edges(np.array(unconnected_vertex_pairs))
    
    unconnected_vertex_pairs_solution = np.array([1]*len(all_edges_after)+[0]*len(unconnected_vertex_pairs))        
    unconnected_vertex_pairs = np.vstack((all_edges_after[:, :2], unconnected_vertex_pairs))
         
    print('Number of unconnected vertex pairs for prediction: ', len(unconnected_vertex_pairs_solution))
    print('Number of vertex pairs that will be connected: ' , sum(unconnected_vertex_pairs_solution))
    print('Ratio of vertex pairs that will be connected: ' , sum(unconnected_vertex_pairs_solution)/len(unconnected_vertex_pairs_solution))

    return get_edges_from_sparse(adjs[0]), unconnected_vertex_pairs, unconnected_vertex_pairs_solution

In [ ]:
def compute_network_matrices(mat):
    all_degs = mat.sum(0).T
    
    l = sparse.linalg.eigsh(mat, k=1, return_eigenvectors=False)
    katz = katz_centrality(nx.Graph(mat), alpha=1/(l[0]+1))
    k = pd.DataFrame.from_dict(katz, orient='index')
    
    mat = mat.dot(mat)
    mat.eliminate_zeros()
    
    all_degs2 = mat.sum(0).T    
    
    return (all_degs, k, mat, all_degs2)
    
def compute_network_properties(mats, vlist):
    """
    Computes hand-crafted properties for all vertices in vlist
    """
    all_degs, k, mat, all_degs2 = mats
    
    all_properties=[]  
    
    d0 = all_degs[vlist[:,0]]
    d1 = all_degs[vlist[:,1]]
    div = np.multiply(d0, d1)
    
    all_properties.append(d0/all_degs.max())
    all_properties.append(d1/all_degs.max())
    all_properties.append(div/(all_degs.max()**2)) #Preferential Attachment

    all_properties.append(k.loc[vlist[:,0]].values)
    all_properties.append(k.loc[vlist[:,1]].values)

    ####
    m = mat[vlist[:,0], vlist[:,1]].T
    all_properties.append(m/m.max()) #Common neighbours

    all_properties.append(np.divide(m, div, out=np.zeros_like(m), where=div!=0)) #Leicht-Holme-Newman Index
    div = np.sqrt(div)
    all_properties.append(np.divide(m, div, out=np.zeros_like(m), where=div!=0)) #Salton Index (cosine similarity)
    div = np.minimum(d0, d1)
    all_properties.append(np.divide(m, div, out=np.zeros_like(m), where=div!=0)) #Hub Promoted Index
    div = np.maximum(d0, d1)
    all_properties.append(np.divide(m, div, out=np.zeros_like(m), where=div!=0)) #Hub Depressed Index
    div = d0 + d1
    all_properties.append(np.divide(m, div, out=np.zeros_like(m), where=div!=0)) #Sørensen Index
    div = d0 + d1 - m
    all_properties.append(np.divide(m, div, out=np.zeros_like(m), where=div!=0)) #Jaccard coefficient
    
    d0 = all_degs2[vlist[:,0]]/all_degs2.max()
    d1 = all_degs2[vlist[:,1]]/all_degs2.max()

    all_properties.append(d0)
    all_properties.append(d1)
    all_properties.append(np.multiply(d0, d1))

    return np.squeeze(np.array(all_properties))

In [ ]:
def create_nx_features(nx_alg, yy, graph, vlist):
    adj = get_sparse_from_year(yy, graph)
    G = nx.Graph(adj)

    p = nx_alg(G, iter(vlist))
    p = pd.DataFrame(p)
        
    return p.loc[:,2].values

In [ ]:
def link_examples_to_features(link_examples, embedding, binary_operator):
    return np.sum(binary_operator(embedding[link_examples[:,0]],
                                   embedding[link_examples[:,1]]), axis=1)

def operator_hadamard(u, v):
    u_norm = np.sqrt(np.sum(np.power(u, 2), axis=1))
    v_norm = np.sqrt(np.sum(np.power(v, 2), axis=1))
    
    return np.multiply(u / u_norm.reshape(-1,1),
                       v / v_norm.reshape(-1,1))

def operator_l1(u, v):
    return np.abs(u - v)

def operator_l2(u, v):
    return (u - v) ** 2/u.shape[1]

def operator_avg(u, v):
    return (u + v) / 2

def operator_concat(u, v):
    return np.concatenate((u,v), axis=-1)

In [ ]:
def compute_embedding_features(yy, edges, bin_ops):
    embed_features = []
    with open(os.path.join(DRIVE_PATH, 'node2vec_embeddings', 'node2vec-'+str(yy) +'.pkl'), "rb") as output_file:
        node_embeddings = pickle.load(output_file) # (64719, 128)
        for bin_op in bin_ops:
            embed_features.append(link_examples_to_features(edges, node_embeddings, bin_op))

    return np.squeeze(np.array(embed_features))

In [ ]:
def save_feature(mat, fname, path=DRIVE_PATH):
    with open(os.path.join(path, 'features', fname+'.npz'), "wb") as f:
        #pickle.dump(mat, f)
        np.savez_compressed(f, a=mat)
    return 0

def load_feature(fname, path=DRIVE_PATH):
    with np.load(os.path.join(path, 'features', fname+'.npz')) as dat:
        return dat['a']

In [ ]:
def generate_train(data, batch_size):
    d0, d1 = data
    while True:
        idx0 = np.random.choice(len(d0), batch_size)
        yield d0[idx0], d1[idx0]
        
def generate_test(data, batch_size):
    data_len = len(data)
    idx_start = 0
    while True:
        idx0 = range(idx_start, min(idx_start+batch_size, data_len))
        idx_start += batch_size

        yield data[idx0]

# Load graph

In [ ]:
full_dynamic_graph_sparse,unconnected_vertex_pairs,year_start,years_delta = pickle.load( open( data_source, "rb" ) )

In [ ]:
edges_used = 3000000#1*10**6 # Best would be to use all vertices, to create more training data. But that takes long and requires huge amount of memory. So here we use a random subset.
vertex_degree_cutoff = 15
_, train_edges_for_checking, train_edges_solution = create_training_data(
    full_dynamic_graph_sparse, 
    year_start-years_delta, 
    years_delta, 
    edges_used=edges_used, 
    vertex_degree_cutoff=vertex_degree_cutoff)

    Create Graph for  2014


/home/ubuntu/anaconda3/envs/tensorflow2_p37/lib/python3.7/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


    num of edges:  1843252
    Create Graph for  2017
    num of edges:  5568238
2883327
Number of unconnected vertex pairs for prediction:  5758889
Number of vertex pairs that will be connected:  2883327
Ratio of vertex pairs that will be connected:  0.5006741751751076


In [ ]:
np.random.seed(0)
idx_col = np.random.choice([0,1], size=train_edges_for_checking.shape[0])
np.random.seed(0)
idx_row = np.random.choice(len(train_edges_for_checking), size=len(train_edges_for_checking), replace=False)
train_edges_for_checking = np.array([train_edges_for_checking[idx_row, idx_col], 
                                     train_edges_for_checking[idx_row, 1-idx_col]]).T

train_edges_solution = train_edges_solution[idx_row]
print(train_edges_for_checking.shape, train_edges_solution.shape)

(5758889, 2) (5758889,)


In [ ]:
np.random.seed(0)
idx_col = np.random.choice([0,1], size=unconnected_vertex_pairs.shape[0])
idx_row = np.arange(len(unconnected_vertex_pairs))
unconnected_vertex_pairs = np.array([unconnected_vertex_pairs[idx_row, idx_col], 
                                     unconnected_vertex_pairs[idx_row, 1-idx_col]]).T
print(unconnected_vertex_pairs.shape)

(1000000, 2)


In [ ]:
save_feature(train_edges_for_checking, 'edges')
save_feature(train_edges_solution, 'train_solutions')
save_feature(unconnected_vertex_pairs, 'unconnected_vertex_pairs')

In [ ]:
train_edges_for_checking[:10], unconnected_vertex_pairs[:10]

(array([[ 1748,  5950],
        [51712, 31782],
        [41359, 20995],
        [53374, 54042],
        [40188, 21390],
        [47402, 33539],
        [36194, 34151],
        [36575, 34650],
        [56348, 33309],
        [45356, 35843]], dtype=int32),
 array([[40338, 18732],
        [ 8023,  4221],
        [ 4769,  2822],
        [39823, 35173],
        [15705, 33319],
        [16627, 40028],
        [ 9671, 39235],
        [28625, 36765],
        [35665, 17213],
        [10643, 25217]], dtype=int32))

# Get network features

In [ ]:
past = 3

In [ ]:
train_years = range(year_start-years_delta, year_start-years_delta-past, -1)
eval_years = range(year_start, year_start-past, -1)

train_features, eval_features = [], []
for yy in sorted(set(list(train_years)+list(eval_years)), reverse=True):      
    adj = get_sparse_from_year(yy, full_dynamic_graph_sparse)
    
    print('year:', yy, 'edges:', adj.count_nonzero()//2)
    
    mats = compute_network_matrices(adj)
    
    if yy in train_years:
        feat = compute_network_properties(mats, train_edges_for_checking)
        train_features.extend(feat)
    
    if yy in eval_years:
        feat = compute_network_properties(mats, unconnected_vertex_pairs)
        eval_features.extend(feat)
        
train_features = np.array(train_features).T
eval_features = np.array(eval_features).T

print(train_features.shape, eval_features.shape)

In [ ]:
save_feature(train_features, 'train_features')
save_feature(eval_features, 'eval_features')

# Get embedding features

In [ ]:
past = 5

In [ ]:
bin_ops = [operator_l2, operator_hadamard]

train_years = range(year_start-years_delta, year_start-years_delta-past, -1)
eval_years = range(year_start, year_start-past, -1)

train_features, eval_features = [], []
for yy in sorted(set(list(train_years)+list(eval_years)), reverse=True):    
    print('year:', yy)
    
    if yy in train_years:
        feat = compute_embedding_features(yy, train_edges_for_checking, bin_ops)
        train_features.extend(feat)
    
    if yy in eval_years:
        feat = compute_embedding_features(yy, unconnected_vertex_pairs, bin_ops)
        eval_features.extend(feat)
        
train_features = np.array(train_features).T
eval_features = np.array(eval_features).T

print(train_features.shape, eval_features.shape)

In [ ]:
save_feature(train_features, 'embed_features')
save_feature(eval_features, 'eval_embed_features')

# Load all features

In [ ]:
train_edges_for_checking = load_feature('edges')
train_edges_solution = load_feature('train_solutions')
unconnected_vertex_pairs = load_feature('unconnected_vertex_pairs')

train_features = load_feature('norm_features')
eval_features = load_feature('norm_eval_features')

embed_features = load_feature('norm_embed_features')
eval_embed_features = load_feature('norm_eval_embed_features')

# Train model

In [ ]:
(   idx_train,
    idx_test,
    labels_train,
    labels_test,
) = train_test_split(range(len(train_edges_solution)), train_edges_solution, train_size=0.95, test_size=0.05)

data_train = np.hstack((train_features[idx_train], embed_features[idx_train]))
data_test = np.hstack((train_features[idx_test], embed_features[idx_test]))
data_eval = np.hstack((eval_features, eval_embed_features))

print(data_train.shape, data_test.shape, data_eval.shape)

In [ ]:
input_shape = (data_train.shape[1],)
dropout_rate = 0.2

model = keras.Sequential([
    keras.layers.Input(shape=input_shape),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(dropout_rate),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(dropout_rate),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(dropout_rate),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=5e-4),
        loss=keras.losses.binary_crossentropy,
        metrics=[keras.metrics.binary_accuracy, keras.metrics.AUC()],
    )
model.summary()

In [ ]:
batch_size = 128

model.fit(generate_train((data_train, labels_train),  batch_size),
          steps_per_epoch = 30000,
          epochs = 50,
          verbose = 2,
          callbacks = [keras.callbacks.EarlyStopping(monitor='loss', patience=1, restore_best_weights=True),
                    keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2, patience=0, min_lr=5e-6)
                    ],
          use_multiprocessing=False
         )

In [ ]:
batch_size = 64

p = model.predict(generate_test(data_test, batch_size), 
                  steps=len(data_test)//batch_size+1, 
                  use_multiprocessing=False)

print(roc_auc_score(labels_test, p))

# Predict on test set

In [ ]:
batch_size = 64

p1 = model.predict(generate_test(data_eval, batch_size), 
                  steps=len(unconnected_vertex_pairs)//batch_size+1, 
                  use_multiprocessing=False)

sorted_predictions_eval=np.flip(np.argsort(p1, axis=0))   
print(p1[sorted_predictions_eval[0]], p1[sorted_predictions_eval[-1]])
print(p1[:15])
print(sum(p1>0.9)/len(unconnected_vertex_pairs)*100)

In [ ]:
submit_file=os.path.join(DRIVE_PATH,"pred"+str(datetime.now())+".json")
all_idx_list_float=list(map(float, sorted_predictions_eval))
with open(submit_file, "w", encoding="utf8") as json_file:
    json.dump(all_idx_list_float, json_file)
print(submit_file)